## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


## Решение:

### Вариант 1 (с pandas):

In [75]:
import pandas as pd

In [97]:
%%time

df = pd.read_csv('ml-latest-small/ratings.csv')
f_data = df.groupby('userId').agg({'rating': 'count', 'timestamp': ['max','min']})
f_data['ToL'] = f_data['timestamp']['max'] - f_data['timestamp']['min']
result = f_data[f_data['rating']['count'] > 100]['ToL'].mean()
result

CPU times: user 56.1 ms, sys: 17.7 ms, total: 73.8 ms
Wall time: 79.2 ms


40080507.4496124

### Вариант 2 (без pandas):

In [98]:
%%time

with open('ml-latest-small/ratings.csv') as f:
    tol_sum = 0
    tol_els = 0
    previous_user = None
    f.readline()
    for line in f:
        userId, a, b, timestamp = line.strip().split(',')
        if previous_user:
            if userId == previous_user:
                ucount += 1
                user_data.append(int(timestamp))
            else:
                if ucount > 100:
                    tol_sum += max(user_data) - min(user_data)
                    tol_els += 1
                user_data = []
                user_data.append(int(timestamp))
                ucount = 1
                previous_user = userId
        else:
            previous_user = userId
            user_data = []
            user_data.append(int(timestamp))
            ucount = 1
tol_sum += max(user_data) - min(user_data)
tol_els += 1
tol_avg = tol_sum / tol_els
tol_avg

CPU times: user 133 ms, sys: 5.65 ms, total: 138 ms
Wall time: 159 ms


40080507.4496124

Интересно, что с использованием pandas получилось в два раза быстрее.

## Задание 2

Дана статистика услуг перевозок клиентов компании по типам:
* rzd - железнодорожные перевозки
* auto - автомобильные перевозки
* air - воздушные перевозки
* client_base - адреса клиентов

Необходимо сформировать две таблицы:

* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [89]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

In [90]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

In [91]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

In [92]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

## Решение:

### Без адресов:

In [93]:
rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


### С адресами:

In [94]:
client_base.merge(rzd, on='client_id', how='outer').merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

## Решение:

### Как записывать:

Первое, что я попробовал бы узнать, — время, когда совершены покупка/визит, и время, когда записаны координаты (если возможно вытащить такую информацию из базы данных). Если известны время, когда пользователь совершил покупку/визит, и время, когда записаны координаты, можно привязать к каждой покупке/визиту ближайшую по времени запись координат (чтобы разница между событиями была, скажем, не больше часа).

Если времени у нас нет, можно проверить количество записей координат для каждого пользователя. Количество записей координат совпадает с количеством визитов/покупок, можно последовательно привязать одно к другому.

Если количество координат больше количества визитов/покупок и время их записи неизвестно, можно попробовать проанализировать, насколько отличаются координаты. Если все координаты пользователя достаточно близки (скажем, отличаются меньше, чем на 0,2 градуса) можно посчитать медиану и записать ее. При этом, если пользователь совершил, к примеру, три визита, можно попробовать записывать координаты, отличающиеся друг от друга более чем на 0,2 градуса в список. 
Примерно так:
* [[(64.211, 43.111)(64.200, 43,100),(...)],
* [(65.202, 45.212),(65.200, 45.201),(...)],
* [(148.75, 42.540),(148.69, 42,541).(...)]]

Если количество элементов в списке совпадет с количеством визитов, можно вычислить медианы каждого набора координат и последовательно сопоставить их с визитами. Хотя тут, конечно, уже очень много «если» и большая доля случайности.

### Что дают:

Если в списке найдутся координаты, которые часто повторяются (у разных пользователей, с каким-то небольшим отклонением), можно проверить, что это за места:
* Может быть, это один из реальных магазинов (пользователи приходят посмотреть на товар, но покупают в интернете)
* Может быть, где-то поблизости размещена реклама (вероятно, она неплохо работает),
* Может быть, что-то еще мотивирует пользователей совершить покупку именно в этом месте.

Если мы работаем в нескольких регионах, можно посчитать распределение пользователей по городам. Возможно, в каком-то городе мы очень популярны, а в в каком-то — нет. Это можно использовать в логистике и маркетинговых кампаниях.

Можно также посчитать распределение пользователей по районам. Возможно, в каких-то районах мы работаем удачно, а в каких-то проваливаемся.